In [ ]:
import sys
import utils
reload(utils)
from utils import *
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/Nicola_model')
from dataset_with_post import PiedmontDataset

In [ ]:
import gzip
import shutil

# Define the input gzipped file and the output file
input_gz_file = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont.tar.gz'
output_file = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont.tar'

# Open the gzipped file and decompress it
with gzip.open(input_gz_file, 'rb') as f_in:
    with open(output_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print(f"File decompressed to {output_file}")


In [ ]:
import tarfile

input_tar_file = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont.tar'
output_directory = '/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont/'

# Open the .tar file and extract its contents
with tarfile.open(input_tar_file, 'r') as tar:
    tar.extractall(path=output_directory)

print(f"Files extracted to {output_directory}")


In [ ]:
fires = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont_2012_2024_fa.geojson')
fires = separate_date(fires, 'initialdate')

for f in fires[fires.YYYY < 2016].id:
    d = f'/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont/fire_{f}/'
    if os.path.exists(d):
        shutil.rmtree(d)

fires[fires.YYYY >= 2016].to_file('/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson', index=False, driver='GeoJSON')

gdf = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson')
test = PiedmontDataset(root_dir='/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont/',
                       geojson_path = '/nfs/home/genovese/thesis-wildfire-genovese/database/piedmont_2016_2024_fa.geojson')

---

Download images for negative fires

In [ ]:
non_fires = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/negative_fires_grid.geojson')

In [ ]:
def reverse_day(year, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

    for index, month in enumerate(days_per_month):
        if (day - month) == 0:
            return (month, index+1)
        elif (day - month) > 0:
            day -= month
        else:
            return (day, index+1)

In [ ]:
non_fires['DD'] = non_fires.apply(lambda x: reverse_day(x['YYYY'], x['day'])[0], axis=1)
non_fires['MM'] = non_fires.apply(lambda x: reverse_day(x['YYYY'], x['day'])[1], axis=1)

from pandas._libs.tslibs.timestamps import Timestamp
non_fires['initialdate'] = non_fires.apply(lambda x: Timestamp(year = x.YYYY, month=x.MM, day=x.DD), axis=1)
non_fires['id'] = [-(idx+10) for idx in non_fires.index]

non_fires['polygon'] = non_fires.buffer(1000)
fires = non_fires.set_geometry(col='polygon')
fires.drop(columns='geometry', inplace=True)
fires.rename(columns={'polygon': 'geometry'}, inplace=True)
fires = fires.to_crs('32632')

fires.to_file('/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson',driver='GeoJSON', index=False)

---

In [ ]:
config_path = "/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/congif.yaml" # Definisci il percorso del config

config_example = {
        "geojson_path": "/nfs/home/genovese/thesis-wildfire-genovese/src/OnFireForest/sentinel_negative_fires/negatives.geojson",
        "interval_pre_fire_days": 7, # AGGIORNATO
        "satellite": "sentinel-2",  # AGGIORNATO
        "patch_size_pixels": 256,
        "target_crs": "EPSG:32632", # NUOVO
        "min_fire_area_sq_m": 200,  # NUOVO
        "image_similarity_tolerance": 0.005 # NUOVO
    }

with open(config_path, "w") as f:
    yaml.dump(config_example, f)
    
print(f"Generato o aggiornato il file di configurazione: {config_path}")


config_data = load_config(config_path) # Carica il config dal percorso specificato

print("\n--- Inizio Processo Batch per TUTTI gli incendi ---")

gdf_all_fires_filtered = gpd.read_file(config_data["geojson_path"])
min_date_for_sentinel2 = datetime(2015, 6, 23) 
    
# # Filtra gli incendi per data (solo quelli dopo l'inizio di Sentinel-2)
# gdf_all_fires_filtered = gdf_all_fires[pd.to_datetime(gdf_all_fires["initialdate"]) >= min_date_for_sentinel2]
    
print(f"Trovati {len(gdf_all_fires_filtered)} incendi nel GeoJSON. Filtrati {len(gdf_all_fires_filtered)} incendi dopo il {min_date_for_sentinel2.strftime('%Y-%m-%d')}.")

for idx, fire_row in tqdm(gdf_all_fires_filtered.iterrows()):
    try:
        process_single_fire(fire_row["id"], config_data, '/nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont')
    except Exception as e:
        print(f"🚨 ERRORE GRAVE durante il processo dell'incendio ID {fire_row['id']}: {e}")
            # Puoi aggiungere qui della logica per registrare gli errori o riprovare

print("\n--- Processo Batch Completato ---")

In [ ]:
from PIL import Image

parent = '/nfs/home/genovese/thesis-wildfire-genovese/database/negative_piedmont'
directories = os.listdir(parent)

black_image = Image.new("RGB", (256, 256), (0, 0, 0))

for fire in directories:
    dir_path = os.path.join(parent, fire)
    if os.path.isdir(dir_path):
        image_path = os.path.join(parent, dir_path, f"{fire}_GTSentinel.png")
        black_image.save(image_path)
        print(f"Saved black image to: {image_path}")
    else:
        print(f"Directory does not exist: {dir_path}")

---